# Задание 2
Генерация синтетических данных с использованием Apache Spark

Цель задания: Использовать Apache Spark для создания синтетического набора данных, который имитирует информацию о покупках в интернет-магазине. Набор данных должен включать в себя информацию о заказах, включая дату заказа, идентификатор пользователя, название товара, количество и цену. Сгенерированные данные будут использованы для последующего анализа покупательской активности и понимания потребительских трендов.
Шаги выполнения:

    Генерация данных:
        Создать DataFrame с полями: Дата, UserID, Продукт, Количество, Цена.
        Данные для поля Продукт генерируются из списка возможных товаров ( не меньше 5 товаров )
        Количество и Цена должны генерироваться случайно в заданных пределах.
        Дата должна быть в пределах последнего года.
        UserID представляет собой случайное число, имитирующее идентификаторы пользователей.
        Обратите внимание, что должна быть возможности изменять количество сгенерированных строк. Минимальное количество - 1000 строк.
    Сохранение данных:
        Сохранить сгенерированный DataFrame в формате CSV для последующего анализа.

Результат выполнения задания (код генерации синтетических данных и созданный файл *.csv) необходимо выложить в github/gitlab и указать ссылку на Ваш репозиторий (не забудьте: репозиторий должен быть публичным).

### Импорт библиотек

In [1]:
#!pip install Faker

In [2]:
import random
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, DoubleType
from datetime import datetime, timedelta

### Инициализация SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Synthetic Data Generation") \
    .getOrCreate()

24/07/22 03:54:20 WARN Utils: Your hostname, marina-linux resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface wlp0s20f3)
24/07/22 03:54:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/22 03:54:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/22 03:54:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Инициализация Faker

In [4]:
fake = Faker('ru_RU')

### Функция для генерации данных

In [5]:
def generate_random_date():
    start_date = datetime.now() - timedelta(days=365)
    random_date = start_date + timedelta(days=random.randint(0, 365))
    return random_date.strftime('%Y-%m-%d')

# Количество строк для генерации
num_rows = int(input("Enter the number of rows to generate: "))

# Список продуктов
products = ['Телефон', 'Ноутбук', 'Планшет', 'Часы', 'Наушники']

# Генерация данных
data = []
for _ in range(num_rows):
    data.append((generate_random_date(),
                 random.randint(1, 1000),  # UserID
                 random.choice(products),  # Продукт
                 random.randint(1, 5),     # Количество
                 round(random.uniform(1000, 50000), 2)))  # Цена

# Создание DataFrame
df = spark.createDataFrame(data, ['Дата', 'UserID', 'Продукт', 'Количество', 'Цена'])

### Сохранение DataFrame в CSV

In [6]:
output_path = 'synthetic_data.csv'
df.coalesce(1).write.csv(output_path, header=True)

# Остановка SparkSession
spark.stop()

print(f"Data saved to {output_path}")

Data saved to synthetic_data.csv
